In [2]:
from __future__ import annotations
from pathlib import Path
from s7.models import create_config
from s7.xlsparser import XLSParser
from s7.classgen import class_factory

# Import Configuration
xls_config = create_config(Path(".").resolve() / 'inhibitor_conf.yml')

# Generate Specific Parser from config
parser = XLSParser(xls_config)
v = parser.get()
print (v)

# Generate entity (not a S7-model yet...)
XLSData = class_factory("XLSData", parser.get())

x = XLSData()
x.inhibitorEfficiency


{'sampleType': ['LiCO3', 'LiCO3', 'LiCl', 'LiCl', 'LiCl'], 'casNumber': ['554-13-2', '554-13-2', '7447-41-8', '7447-41-8', '7447-41-8'], 'runID': ['27082021_1mMLiCO3_0.1MNaCl_AA2024_run4', '27082021_1mMLiCO3_0.1MNaCl_AA2024_run5', '11082021_1mMLiCl_0.1MNaCl_AA2024_run1', '11082021_1mMLiCl_0.1MNaCl_AA2024_run2', '11082021_1mMLiCl_0.1MNaCl_AA2024_run3'], 'inhibitorEfficiency': ['82.84807910472972', '82.70578220798093', '-32.10177835991128', '0.7790838781530862', '31.69712512083033'], 'impedance_log_24h': ['5.00314131824638', '4.99955315940226', '4.11654541915057', '4.24085085066989', '4.40301509970376'], 'impedance_ohm_24h': ['100725.93749999841', '99897.1640624986', '13078.123046874862', '17412.08789062481', '25293.859374999603'], 'impedance_kohm_24h': ['79.06986093749876', '78.4192737890614', '10.266326591796767', '13.668488994140477', '19.85567960937469']}


['82.84807910472972',
 '82.70578220798093',
 '-32.10177835991128',
 '0.7790838781530862',
 '31.69712512083033']

In [6]:
from pathlib import Path
from s7.entitygen import create_entity
from s7.models import create_config

xls_config = create_config(Path(".").resolve() / 'inhibitor_conf.yml')

DataSourceEntity = create_entity(
    data_model=Path(".").resolve() / "efficiency.yaml",
    resource_config={
        "downloadUrl": (Path(".").resolve() / "Inhibitor_Data_Analysis_summary.xlsx").as_uri(),
        "mediaType": "application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        "configuration": xls_config,
    },
)
entity = DataSourceEntity()
# entity.schema()
# help(entity)
entity.casNumber

False


ApiError: APIError: status=500 Cannot initialize data resource: session_id='session-97854d2e-ec99-4e82-9e82-677185d4e50f' resource_id='dataresource-e4d5c4b7-431e-41b5-bf48-bc3860c8d093'

In [ ]:
from s7.graph import Graph
g = Graph()
d = XLSData()

print (d.sampleType)

g.append(('entity', 'hasProperty', 'entity:sampleType'))
g.append(('entity:sampleType', 'propertyOf', 'entity'))

g.append(('entity', 'hasProperty', 'entity:casNumber'))
g.append(('entity:casNumber', 'propertyOf', 'entity'))

g.append(('entity:sampleType', 'mapsTo', 'ontology:sampleType'))
g.append(('entity:sampleType', 'propertyOf', 'entity'))

g.append(('ontology:sampleType', 'mapsfrom', 'entity:sampleType'))
g.append(('entity:casNumber', 'mapsTo', 'ontology:casNumber'))
g.append(('ontology:casNumber', 'mapsFrom', 'entity:casNumber'))
g.append(('entity:inhibitorEfficiency', 'mapsTo', 'ontology:inhibitorEfficiency'))
g.append(('ontology:inhibitorEfficiency', 'mapsTo', 'entity:inhibitorEfficiency'))

g.append(((lambda : d.sampleType), 'executes', 'entity:sampleType'))
g.append(((lambda : d.casNumber), 'executes', 'entity:casNumber'))
g.append(((lambda : d.inhibitorEfficiency), 'executes', 'entity:inhibitorEfficiency'))

# consumer mapping
g.append(('consumer:eff', 'mapsTo', 'ontology:inhibitorEfficiency'))
g.append(('consumer:cas', 'mapsTo', 'ontology:casNumber'))


In [ ]:


def path(origin, destination, link=None, visited=[]):
    if not link:
        link = [origin]    
    visited.append(origin)
    for _, r, o in g.match(origin, None, None):
        if not o in visited:
            if o == destination:                
                return link
            else:
                link.append(o)            
                return path(o, destination, visited)
    
link = path('consumer:cas', 'entity')
print (link)

In [ ]:
x.impedance

In [ ]:
import math
math.pow(10, x.impedance[0])